<a href="https://colab.research.google.com/github/probml/pyprobml/blob/master/notebooks/word_analogies_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Solving word analogies using pre-trained word embeddings

Based on D2L 14.7

http://d2l.ai/chapter_natural-language-processing-pretraining/similarity-analogy.html



In [1]:
import numpy as np
import matplotlib.pyplot as plt
np.random.seed(seed=1)
import math

import os
import random
import torch
from torch import nn
from torch.nn import functional as F

!mkdir figures # for saving plots

!wget https://raw.githubusercontent.com/d2l-ai/d2l-en/master/d2l/torch.py -q -O d2l.py
import d2l

# Get pre-trained word embeddings

Pretrained embeddings taken from 

GloVe website: https://nlp.stanford.edu/projects/glove/

fastText website: https://fasttext.cc/


In [3]:
d2l.DATA_HUB['glove.6b.50d'] = (d2l.DATA_URL + 'glove.6B.50d.zip',
                                '0b8703943ccdb6eb788e6f091b8946e82231bc4d')


d2l.DATA_HUB['glove.6b.100d'] = (d2l.DATA_URL + 'glove.6B.100d.zip',
                                 'cd43bfb07e44e6f27cbcc7bc9ae3d80284fdaf5a')


d2l.DATA_HUB['glove.42b.300d'] = (d2l.DATA_URL + 'glove.42B.300d.zip',
                                  'b5116e234e9eb9076672cfeabf5469f3eec904fa')


d2l.DATA_HUB['wiki.en'] = (d2l.DATA_URL + 'wiki.en.zip',
                           'c1816da3821ae9f43899be655002f6c723e91b88')

In [4]:
class TokenEmbedding:
    """Token Embedding."""
    def __init__(self, embedding_name):
        self.idx_to_token, self.idx_to_vec = self._load_embedding(
            embedding_name)
        self.unknown_idx = 0
        self.token_to_idx = {
            token: idx for idx, token in enumerate(self.idx_to_token)}

    def _load_embedding(self, embedding_name):
        idx_to_token, idx_to_vec = ['<unk>'], []
        data_dir = d2l.download_extract(embedding_name)
        # GloVe website: https://nlp.stanford.edu/projects/glove/
        # fastText website: https://fasttext.cc/
        with open(os.path.join(data_dir, 'vec.txt'), 'r') as f:
            for line in f:
                elems = line.rstrip().split(' ')
                token, elems = elems[0], [float(elem) for elem in elems[1:]]
                # Skip header information, such as the top row in fastText
                if len(elems) > 1:
                    idx_to_token.append(token)
                    idx_to_vec.append(elems)
        idx_to_vec = [[0] * len(idx_to_vec[0])] + idx_to_vec
        return idx_to_token, torch.tensor(idx_to_vec)

    def __getitem__(self, tokens):
        indices = [
            self.token_to_idx.get(token, self.unknown_idx)
            for token in tokens]
        vecs = self.idx_to_vec[torch.tensor(indices)]
        return vecs

    def __len__(self):
        return len(self.idx_to_token)

Get a 50dimensional glove embedding, with vocab size of 400k

In [5]:
glove_6b50d = TokenEmbedding('glove.6b.50d')

In [6]:
len(glove_6b50d)

400001

Map from word to index and vice versa.

In [7]:
glove_6b50d.token_to_idx['beautiful'], glove_6b50d.idx_to_token[3367]

(3367, 'beautiful')

In [50]:
embedder = glove_6b50d

In [16]:
#embedder = TokenEmbedding('glove.6b.100d')

In [56]:
embedder.idx_to_vec.shape

torch.Size([400001, 50])

# Finding most similar words

In [58]:
def knn(W, x, k):
    # The added 1e-9 is for numerical stability
    cos = torch.mv(W, x.reshape(-1,)) / (
      (torch.sqrt(torch.sum(W * W, axis=1) + 1e-9) * torch.sqrt((x * x).sum())) )
    _, topk = torch.topk(cos, k=k)
    return topk, [cos[int(i)] for i in topk]

In [9]:
def get_similar_tokens(query_token, k, embed):
    topk, cos = knn(embed.idx_to_vec, embed[[query_token]], k + 1)
    for i, c in zip(topk[1:], cos[1:]):  # Remove input words
        print(f'cosine sim={float(c):.3f}: {embed.idx_to_token[int(i)]}')

In [59]:
get_similar_tokens('man', 3, embedder)

cosine sim=0.886: woman
cosine sim=0.856: boy
cosine sim=0.845: another


In [52]:
get_similar_tokens('banana', 3, embedder)

cosine sim=0.815: bananas
cosine sim=0.787: coconut
cosine sim=0.758: pineapple


# Word analogies

In [46]:
# We slightly modify D2L code so it works on the man:woman:king:queen example
def get_analogy(token_a, token_b, token_c, embed):
    vecs = embed[[token_a, token_b, token_c]]
    x = vecs[1] - vecs[0] + vecs[2]
    topk, cos = knn(embed.idx_to_vec, x, 10)
    # remove word c from nearest neighbor
    idx_c = embed.token_to_idx[token_c]
    topk = list(topk.numpy())
    topk.remove(idx_c)
    return embed.idx_to_token[int(topk[0])]  

In [53]:
get_analogy('man', 'woman', 'king', embedder)

'queen'

In [54]:
get_analogy('man', 'woman', 'son', embedder)

'daughter'

In [55]:
get_analogy('beijing', 'china', 'tokyo', embedder)

'japan'